# **Linear Regression on Bike Sharing**

<hr>

**Storyline**

RentoBikes,  a leading US bike-sharing provider, is addressing pandemic-induced revenue challenges by analyzing factors influencing bike demand. Engaging a consulting firm, they aim to predict demand post-lockdown and tailor their strategy. Data preparation involves converting numeric variables and retaining the 'yr' column for its predictive value. The model, focusing on the 'cnt' variable, aims to reveal insights into demand dynamics, guiding BoomBikes in optimizing their services for post-Covid market recovery

<hr>


<hr>

# **Stage 1: Reading and Understanding the data**

In [ ]:
# Libraries needed

import warnings
import numpy as np
import pandas as pd
import seaborn as sns
warnings.filterwarnings('ignore')
